In [1]:
import os, time, shutil
from pathlib import Path
from glob import glob
import numpy as np
import spikeinterface.full as si

In [2]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 5.5
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False
print(sorter_params)

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/phornauer/Git/Kilosort_2020b
{'detect_threshold': 5.5, 'projection_threshold': [10, 4], 'preclust_threshold': 8, 'car': True, 'minFR': 0.01, 'minfr_goodchannels': 0.01, 'nblocks': 5, 'sig': 20, 'freq_min': 150, 'sigmaMask': 30, 'nPCs': 3, 'ntbuff': 64, 'nfilt_factor': 4, 'NT': None, 'do_correction': False, 'wave_length': 61, 'keep_good_only': False, 'n_jobs': -1, 'total_memory': None, 'chunk_size': None, 'chunk_memory': None, 'chunk_duration': '1s', 'progress_bar': True}


In [6]:
input_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/'
path_list  = glob(input_path + '/M*/Network/*/data.raw.h5')
path_list = path_list[1]

['/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M05854/Network/000018/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04694/Network/000015/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04694/Network/000016/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04163/Network/000023/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04163/Network/000022/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04258/Network/000026/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04258/Network/000025/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M05867/Network/000019/data.raw.h5',
 '/net/b

In [18]:
path_list 

['/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04694/Network/000016/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04163/Network/000022/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04258/Network/000025/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M05867/Network/000020/data.raw.h5']

In [4]:
path_parts = path_list[0].split('/')
path_parts[0] = '/'
path_parts[6] = 'intermediate_data'
path_parts[7] = 'MaxTwo'
path_parts[8] = 'phornauer'
save_root = os.path.join(*path_parts[0:12])
save_root

'/net/bs-filesvr02/export/group/hierlemann/intermediate_data/MaxTwo/phornauer/iNeurons/230302/M05854'

In [19]:
for p in path_list:
    print(p)
    path_parts = p.split('/')
    path_parts[0] = '/'
    path_parts[6] = 'intermediate_data'
    save_root = os.path.join(*path_parts[0:12])
    for w in range(6):
        stream_name = 'well00' + str(w)
        save_path = os.path.join(save_root,stream_name)
        output_folder = Path(os.path.join(save_path, 'sorted'))
        
        if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
        #if not os.path.exists(output_folder):
            print(f"\n\n---------Sorting recording {os.path.join(path_parts[10],path_parts[11])} well {w+1}---------\n\n")
            output_folder.mkdir(parents=True, exist_ok=True)
            raw_file = os.path.join(output_folder,'recording.dat')
            wh_file = os.path.join(output_folder, 'temp_wh.dat')
            try:
                rec = si.MaxwellRecordingExtractor(p,stream_name=stream_name)
            except Exception:
                continue
            print(f"DURATION: {rec.get_num_frames() / rec.get_sampling_frequency()} s -- "
                  f"NUM. CHANNELS: {rec.get_num_channels()}")
            try:
                t_start_sort = time.time()
                sorting = si.run_sorter(sorter, rec, output_folder=output_folder, verbose=True,
                                        **sorter_params)
                print(f"\n\nSpike sorting elapsed time {time.time() - t_start_sort} s")
                if os.path.exists(wh_file):
                    os.remove(wh_file)
                if os.path.exists(raw_file):
                    os.remove(raw_file)
            except Exception:
                if os.path.exists(wh_file):
                    os.remove(wh_file)
                if os.path.exists(raw_file):
                    os.remove(raw_file)
                continue

/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/mpriouret/iNeurons/230302/M04694/Network/000016/data.raw.h5


---------Sorting recording 230302/M04694 well 1---------


DURATION: 600.02 s -- NUM. CHANNELS: 1020
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptnafr0wze/script.sh


write_binary_recording:   0%|          | 0/601 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscript0mxzfbtk/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptzzc8nwbn/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/mpriouret/iNeurons/230302/M04694/well000/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   5s. Loading raw data and applying filters... 

Time  61s. Finished preprocessing 92 batches. 

D

write_binary_recording:   0%|          | 0/601 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptstx1i662/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscript4wra1161/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/mpriouret/iNeurons/230302/M04694/well001/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   4s. Loading raw data and applying filters... 

Time  65s. Finished preprocessing 92 batches. 

D

...

RUNNING SHELL SCRIPT: /tmp/tmp_shellscripth8gbxvqm/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptitmx5sjt/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/mpriouret/iNeurons/230302/M05867/well003/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   5s. Loading raw data and applying filters... 

Time  62s. Finished preprocessing 92 batches. 

D

write_binary_recording:   0%|          | 0/601 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscript668gm990/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptfsmm7niz/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/mpriouret/iNeurons/230302/M05867/well004/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   4s. Loading raw data and applying filters... 

Time  55s. Finished preprocessing 92 batches. 

D

write_binary_recording:   0%|          | 0/601 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscript3s12fdfb/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptwukgw5u1/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/mpriouret/iNeurons/230302/M05867/well005/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   5s. Loading raw data and applying filters... 

Time  61s. Finished preprocessing 92 batches. 

D

In [40]:
# 17 
for p in path_list:
    path_parts = p.split('/')
    path_parts[0] = '/'
    path_parts[6] = 'intermediate_data'
    save_root = os.path.join(*path_parts[0:12])
    failed_sortings = []
    for w in range(6):

        stream_name = 'well00' + str(w)
        save_path = os.path.join(save_root,stream_name)

        output_folder = Path(os.path.join(save_path, 'sorted'))
        if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
            print(output_folder)

/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_3/221228/M05867/well001/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_3/221228/M05856/well002/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_3/221228/M05865/well004/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_1/221216/M05093/well001/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_1/221214/M05865/well004/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_1/221214/M05856/well002/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_1/221214/M04258/well002/sorted
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/SCR_rebuttal_week_1/221214/M04258/well004/sorted
